In [1]:
import cv2
import numpy as np
import time
from helper_functions import draw_flow


def main(video_path, show_fps=False, use_roi=True, scale_factor=1.0):
    global last_frame
    print('Starting')

    # Pull the conversion factor from conv_factor.txt if exists
    try:
        with open('conv_factor.txt', 'r') as f:
            conv_factor = float(f.read())
            print(f"Conversion factor found: {conv_factor}")
    except FileNotFoundError:
        print("No conversion factor found. Using default value of 1.0")
        conv_factor = 1.0

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Get FPS from the video file
    real_time_fps = cap.get(cv2.CAP_PROP_FPS)

    ret, prev = cap.read()
    if not ret:
        print("Failed to grab the frame")
        return

    frame_height, frame_width = prev.shape[:2]

    # Calculate the ROI based on scale factor
    w_roi = int(frame_width * scale_factor)
    h_roi = int(frame_height * scale_factor)
    x_roi = (frame_width - w_roi) // 2
    y_roi = (frame_height - h_roi) // 2

    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    frame_count = 0
    start_time = time.time()
    prev_avg_velocity = 0
    winsize = 200

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        if use_roi:
            roi_frame = frame[y_roi:y_roi+h_roi, x_roi:x_roi+w_roi]
        else:
            roi_frame = frame

        gray = cv2.cvtColor(roi_frame, cv2.COLOR_BGR2GRAY)

        # Ensure prev_gray is only set after the first frame is captured
        if frame_count == 1:
            prev_gray = gray

        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale=0.5, levels=1, winsize=winsize, iterations=5, poly_n=5, poly_sigma=1.2, flags=0)
    
        prev_gray = gray.copy()  # Update prev_gray for the next iteration

        vis, prev_avg_velocity = draw_flow(gray, flow, winsize, real_time_fps, show_fps, frame_count, start_time, prev_avg_velocity, conv_factor=conv_factor,color_map_on=True)
        cv2.imshow('Optical flow', vis)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #capture the frame
            last_frame = frame
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "video.mp4"  # Replace with the path to your video file
    main(video_path, show_fps=True, use_roi=True, scale_factor=.5)  # Adjust scale_factor as needed


Starting
Conversion factor found: 0.005714285714285714


In [2]:
last_frame = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)

NameError: name 'last_frame' is not defined